How weather is impacting the electricity demand in Victoria?
1.	Check if there is a correlation between Max temperature and Electricity demand for CITIPOWER distributor.
H1 : When Max temperature is > T_Median electricity demand is > D_Median
H0 : No such relationship

2.	Do the similar analysis for Rain and Solar

In [76]:
import pandas as pd
import dask.dataframe as dd
import datetime
import glob

In [77]:
def refactor_weather_data(bomDF,year):    
    dateslist = pd.date_range(start=f'1/1/{year}', end=f'31/12/{year}')
    weatherlist  = bomDF['Jan'].tolist()
    if isleapyear(year):
        weatherlist = weatherlist + bomDF['Feb'].tolist()[0:29]
    else:
        weatherlist = weatherlist + bomDF['Feb'].tolist()[0:28]          
    weatherlist = weatherlist + bomDF['Mar'].tolist() + bomDF['Apr'].tolist()[0:30] + bomDF['May'].tolist() + bomDF['Jun'].tolist()[0:30] + bomDF['Jul'].tolist() + bomDF['Aug'].tolist() + bomDF['Sep'].tolist()[0:30] + bomDF['Oct'].tolist() + bomDF['Nov'].tolist()[0:30] + bomDF['Dec'].tolist()

    weatherDF = pd.DataFrame({'Date': dateslist, 'Measure': weatherlist})
    return weatherDF


In [78]:
def isleapyear(n):
    if n%4==0 and n%100!=0:
        if n%400==0:
            return True
    elif n%4!=0:
        return False
    


In [79]:

dir = 'BOM/Resources/Weather/_*.csv'
files = glob.glob(dir)
weatherDF = pd.DataFrame()
for f in files:
    filenamelist = f.split("_")
    df = refactor_weather_data(pd.read_csv(f),int(filenamelist[2]))
    df['Distributor'] = filenamelist[1] 
    if filenamelist[3] == '122':
        df['MeasurementType'] = 'MaxTemp'
    elif filenamelist[3] == '123':
        df['MeasurementType'] = 'MinTemp'
    elif filenamelist[3] == '136':
        df['MeasurementType'] = 'Rain Fall'
    elif filenamelist[3] == '193':
        df['MeasurementType'] = 'Solar Exposure'          
    weatherDF = weatherDF.append(df,ignore_index=True)
weatherDF.to_csv('BOM/Resources/Weather/weather.csv')